In [1]:
import graphlab as gl
import graphlab.aggregate as agg

In [11]:
edges = gl.load_sframe("yelp_dataset_challenge_academic_dataset/edges/")
users = gl.load_sframe("yelp_dataset_challenge_academic_dataset/users/")

In [12]:
uf_graph = gl.SGraph(users, edges, 'user_id', 'user_id', 'friend_id')

In [13]:
cc = gl.connected_components.create(uf_graph)

+-----------------------------+

| Number of components merged |

+-----------------------------+

| 710693                      |

| 0                           |

+-----------------------------+

In [14]:
cc.summary()

Class                                   : ConnectedComponentsModel

Graph
-----
num_edges                               : 3563817
num_vertices                            : 552339

Results
-------
graph                                   : SGraph. See m['graph']
component size                          : SFrame. See m['component_size']
number of connected components          : 306530
vertex component id                     : SFrame. See m['component_id']

Metrics
-------
training time (secs)                    : 21.1412

Queryable Fields
----------------
graph                                   : A new SGraph with the color id as a vertex property
component_id                            : An SFrame with each vertex's component id
component_size                          : An SFrame with the size of each component
training_time                           : Total training time of the model



In [18]:
cc_df = cc.component_id

In [19]:
cc_df.head()

__id,component_id
SzfxLfcdvL-UnY9KBR3cUw,64320
nk9281cL4ADcqIp_y3thTA,64320
CT6Sx40eEn4zlSNdepAKFQ,2
IAtkt8yfVFFbb-9wN7-boQ,64320
-CfCLM_crWL131qlh9o5yQ,4
GgRZhdnQqOwiBNnYffW9yQ,64320
veujMq0R4iWIvu8IszIOrw,6
XJHromH0ZXki0YX7goJ0Qw,64320
2DXlHdiBXue4BxExhfo76Q,8
bkycfFGavEkva6AVQWxIDQ,9


#### Let's find the distribution of the number of vertices in each component and see if it makes any sense 

In [23]:
component_counts = cc_df.groupby("component_id", operations={'count':agg.COUNT()})

In [26]:
component_counts.head()

component_id,count
368497,1
251434,1
211023,1
21855,1
233270,1
88004,1
79732,1
366015,1
461249,1
63664,1


In [25]:
component_counts[component_counts>1]

Canvas is accessible via web browser at the URL: http://localhost:63429/index.html
Opening Canvas in default web browser.


In [30]:
component_counts['count'].sketch_summary()


+--------------------+---------------+----------+
|        item        |     value     | is exact |
+--------------------+---------------+----------+
|       Length       |     306530    |   Yes    |
|        Min         |      1.0      |   Yes    |
|        Max         |    241659.0   |   Yes    |
|        Mean        |  1.8019084592 |   Yes    |
|        Sum         |    552339.0   |   Yes    |
|      Variance      | 190514.459677 |   Yes    |
| Standard Deviation | 436.479621147 |   Yes    |
|  # Missing Values  |       0       |   Yes    |
|  # unique values   |       8       |    No    |
+--------------------+---------------+----------+

Most frequent items:
+-------+--------+------+-----+----+----+---+---+--------+
| value |   1    |  2   |  3  | 4  | 5  | 6 | 8 | 241659 |
+-------+--------+------+-----+----+----+---+---+--------+
| count | 302898 | 3220 | 327 | 65 | 15 | 3 | 1 |   1    |
+-------+--------+------+-----+----+----+---+---+--------+

Quantiles: 
+-----+-----+-----+

This don't seem like they are accurate representations of how many **distinct groups of friends** are there

### Let's do a count of degrees

In [36]:
dc = gl.degree_counting.create(uf_graph)

In [43]:
vertices = dc.graph.get_vertices()

In [44]:
vertices.head()

__id,in_degree,out_degree,total_degree
SzfxLfcdvL-UnY9KBR3cUw,3,3,6
nk9281cL4ADcqIp_y3thTA,1,1,2
CT6Sx40eEn4zlSNdepAKFQ,0,0,0
IAtkt8yfVFFbb-9wN7-boQ,1,1,2
-CfCLM_crWL131qlh9o5yQ,0,0,0
GgRZhdnQqOwiBNnYffW9yQ,7,7,14
veujMq0R4iWIvu8IszIOrw,0,0,0
XJHromH0ZXki0YX7goJ0Qw,1,1,2
2DXlHdiBXue4BxExhfo76Q,0,0,0
bkycfFGavEkva6AVQWxIDQ,0,0,0


In [47]:
vertices['total_degree'].sketch_summary()


+--------------------+---------------+----------+
|        item        |     value     | is exact |
+--------------------+---------------+----------+
|       Length       |     552339    |   Yes    |
|        Min         |      0.0      |   Yes    |
|        Max         |     7624.0    |   Yes    |
|        Mean        |  12.904455416 |   Yes    |
|        Sum         |   7127634.0   |   Yes    |
|      Variance      | 5856.27189973 |   Yes    |
| Standard Deviation | 76.5262824115 |   Yes    |
|  # Missing Values  |       0       |   Yes    |
|  # unique values   |      914      |    No    |
+--------------------+---------------+----------+

Most frequent items:
+-------+--------+-------+-------+-------+-------+-------+------+------+------+
| value |   0    |   2   |   4   |   6   |   8   |   10  |  12  |  14  |  16  |
+-------+--------+-------+-------+-------+-------+-------+------+------+------+
| count | 302898 | 66303 | 35246 | 23212 | 16581 | 12454 | 9995 | 8006 | 6725 |
+------

In [54]:
# Vertices where in degree != out degree, implying directed graph
print vertices[vertices['out_degree']!=vertices['in_degree']].head()
print "Number of vertices where in-degree != out-degree: {}".format(len(vertices[vertices['out_degree']!=vertices['in_degree']]))

+------------------------+-----------+------------+--------------+
|          __id          | in_degree | out_degree | total_degree |
+------------------------+-----------+------------+--------------+
| xV1BqbDyrFAJ5FZ9FeHPNg |     5     |     6      |      11      |
| ckLyJAIbb4Do-ymu8fwMQQ |    145    |    146     |     291      |
| T15oL2E_-k6A4y4q_dRkjQ |     15    |     16     |      31      |
| _fpw5dI0BsvXWyELM0CZbQ |     65    |     67     |     132      |
| vyNPMOMztvBrKX9EYfjcZQ |     12    |     11     |      23      |
| UvKK1z2zTSxljes3taL-DQ |     38    |     37     |      75      |
| Qx3Xip71upfc6X3gfmMm_g |     17    |     18     |      35      |
| r2C9y_0eVE6G80q0GxW8KQ |     80    |     81     |     161      |
| Pkn-5urM_rHas9MMSAncCA |     55    |     54     |     109      |
| VVP-DVzNITDwFiMyIad6XQ |     14    |     13     |      27      |
+------------------------+-----------+------------+--------------+
[10 rows x 4 columns]

Number of vertices where in-degree != o

In [53]:
## This explains why the weakly connected components algorithm did not really work here. 

## Using The SNAP.py library

Let's use a community detection method from the snap.py library [here](https://snap.stanford.edu/snappy/). This is a very highly optimized library for doing analysis of networks in python

#### Approach

We will use an approximate community detection method to find distinct groups of friends. 

#### Changing the dataset 

- We can safely remove users with no friends
- Make this graph an undirected graph
- convert user_ids to numbers (as required by the snap library)



In [3]:
import snap

In [65]:
#Choosing vertices with non zero degree
vertices = vertices[vertices['total_degree']>1]

In [68]:
v_index = gl.SArray(range(1, len(vertices)+1))

In [71]:
vertices = vertices.add_column(v_index, name='index')

In [81]:
edges.add_columns([gl.SArray([0]*len(edges)), gl.SArray([0]*len(edges))], ['user_num', 'friend_num'])

user_id,friend_id,user_num,friend_num
18kPq7GPye-YQ3LyKyAZPw,rpOyqD_893cqmDAtJLbdog,0,0
18kPq7GPye-YQ3LyKyAZPw,4U9kSBLuBDU391x6bxU-YA,0,0
18kPq7GPye-YQ3LyKyAZPw,fHtTaujcyKvXglE33Z5yIw,0,0
18kPq7GPye-YQ3LyKyAZPw,8J4IIYcqBlFch8T90N923A,0,0
18kPq7GPye-YQ3LyKyAZPw,wy6l_zUo7SN0qrvNRWgySw,0,0
18kPq7GPye-YQ3LyKyAZPw,HDQixQ-WZEV0LVPJlIGQeQ,0,0
18kPq7GPye-YQ3LyKyAZPw,T4kuUr_iJiywOPdyM7gTHQ,0,0
18kPq7GPye-YQ3LyKyAZPw,z_5D4XEIlGAPjG3Os9ix5A,0,0
18kPq7GPye-YQ3LyKyAZPw,i63u3SdbrLsP4FxiSKP0Zw,0,0
18kPq7GPye-YQ3LyKyAZPw,pnrGw4ciBXJ6U5QB2m0F5g,0,0


In [89]:
# Converted to a pandas data frame with the original id as index for quick lookup
vdf = vertices['__id', 'index'].to_dataframe().set_index("__id")

In [99]:
# Updating the user_num and friend_num columns with the numeric id
edges['user_num'] = vdf.loc[edges['user_id']]['index']
edges['friend_num'] = vdf.loc[edges['friend_id']]['index']

#### Creating the SNAP.py undirected graph

In [111]:
edges.save("yelp_dataset_challenge_academic_dataset/snap_edges")
vertices.save("yelp_dataset_challenge_academic_dataset/snap_vertices")

In [2]:
edges = gl.load_sframe("yelp_dataset_challenge_academic_dataset/snap_edges/")
vertices = gl.load_sframe("yelp_dataset_challenge_academic_dataset/snap_vertices/")

This non-commercial license of GraphLab Create is assigned to mkkedia@dons.usfca.edu and will expire on May 31, 2017. For commercial licensing options, visit https://dato.com/buy/.


[INFO] graphlab.cython.cy_server: GraphLab Create v1.10 started. Logging: /tmp/graphlab_server_1465130929.log


In [5]:
ug = snap.TUNGraph.New(len(vertices), len(edges)/2)

In [6]:
## Add all the nodes
for i in range(len(vertices)):
    ug.AddNode(vertices['index'][i])

In [ ]:
## Add Edges
for i in range(len(edges)):
    ug.AddEdge(int(edges['user_num'][i]),int(edges['friend_num'][i]))

In [11]:
for u,f in zip(edges['user_num'],edges['friend_num']):
    ug.AddEdge(u,int(f))

ValueError: cannot convert float NaN to integer

RuntimeError: Invalid checksum reading 'yelp_dataset_challenge_academic_dataset/snap_ug_176255.graph'.

NameError: name 'i' is not defined